In [1]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
import MDAnalysis
import urllib.request
import yaml
import random


# This defines the path for the NMRlipids databank on your computer. 
# Default is that this repository and the NMRlipids databank repository are cloned to the same folder.
# If this is not the case, change this to the folder where the NMRlipids databank repository is located.
databankPath =  '../../Databank/'

# This enables the access to functions defined in the NMRlipids databank.
sys.path.insert(1, databankPath + '/Scripts/BuildDatabank/')
from databankLibrary import * 

# This initializes the databank and stores the information of all simulations into a list.
# Each list item contains the information from README.yaml file of the given simulation.
systems = initialize_databank(databankPath)


/home/osollila/.conda/envs/py39/lib/python3.9/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


In [5]:
#InterestingComposition = ['POPC', 'CHOL','POPE','POPS','SAPI']
InterestingComposition = ['POPS', 'POPE','POPC']
aplArray = []
interestArray = []

for system in systems:
    skip = True
    
    for interest in InterestingComposition:
        if interest in system['COMPOSITION']:
            skip = False
    
    if skip:
        continue

        
    interestArrayTMP = []
    for interest in InterestingComposition:
        try:
            interestArrayTMP.append(sum(system['COMPOSITION'][interest]['COUNT']) / GetNlipids(system))
        except:
            interestArrayTMP.append(0)
    
    aplArray.append(CalcAreaPerMolecule(system))
    
    interestArray.append(interestArrayTMP)
    
    #print(system['COMPOSITION'])
    #print(CalcAreaPerMolecule(system))
    #print()

In [6]:
from sklearn.linear_model import LinearRegression

X = np.array(interestArray)
y = np.array(aplArray)

# Create an instance of the LinearRegression class
reg = LinearRegression()
 
# Fit the model to the data
reg.fit(X, y)
 
# Print the coefficients of the model
print(reg.coef_)

[20.5685181  24.57644893 26.34993358]


In [8]:
#reg.predict([[0.35, 0.25,0.25,0.1,0.05]])
reg.predict([[0.7, 0.25,0.05]])

array([58.11222545])